In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
plt.style.use('ggplot')

In [81]:
# Pull in dictionaries from other notebooks
%store -r boa_dict
%store -r wells_dict
%store -r chase_dict
%store -r USB_dict
%store -r LD_dict
%store -r cal_dict
%store -r fair_dict

In [51]:
chase_dict['Lender'] = 'JPMorgan Chase'

* Notes to include: 
    * Certain protected classes are not yet represented in the model because there is not enough data to confidently determine error rates. Additionally, it is unclear as to how the data was determined for certain categories. For example, derived sex: joint. Looking at the raw data, there are several applicants (though very few) that chose both male and female on the application. However, there are a much larger number of these applicants in the derived column. In a nutshell, the numbers don't line up. need to check to see if sex was visually observed for many of those values. 
    * Pacific Islanders, native americans, were not compared because there were so few applicants
    * Rather than show the threshold value, maybe "curve" the likelihood of approval so it's more easily interpretable. An 89% probability of approval might sound good, but if the threshold is 81%, that's cutting it a little close. How to curve? Proportion?
* Model inputs are different shapes. Ideas to address: 
    * Wrap model preparation in a function for each bank
    * pull record from DF instead of X_test
    * Run record through model preparation function
    * pass into predict
    * This should help with the User Input issue also

In [52]:
dicts = [boa_dict, wells_dict, chase_dict, USB_dict, LD_dict, cal_dict, fair_dict]
bank_performance = pd.DataFrame(dicts, columns=['Lender', 'Likelihood of Approval', 'Certainty of Prediction', 
                                         'Approval Threshold', 'Black applicant error rate', 
                                         'Asian applicant error rate', 'Hispanic applicant error rate', 
                                         'Female applicant error rate', 'Fossil Fuel Funding', 'Climate Policy Score'])

In [53]:
bank_performance

,Lender,Likelihood of Approval,Certainty of Prediction,Approval Threshold,Black applicant error rate,Asian applicant error rate,Hispanic applicant error rate,Female applicant error rate,Fossil Fuel Funding,Climate Policy Score
0,Bank of America,NaN,0.942305,NaN,-1.297498,0.128949,3.786079,-0.867824,NaN,NaN
1,Wells Fargo,NaN,0.985232,NaN,2.556237,0.692595,0.221873,0.041473,NaN,NaN
2,JPMorgan Chase,NaN,0.960042,NaN,0.000000,-1.604278,6.320217,0.141044,NaN,NaN
3,U.S. Bank,NaN,0.921860,NaN,5.970149,0.228311,-0.196502,-0.572129,NaN,NaN
4,Loan Depot,NaN,0.952986,NaN,0.801833,0.000000,0.046721,-0.039880,NaN,NaN
5,Caliber Home Loans,NaN,0.970476,NaN,0.810185,0.148810,-0.277740,0.087544,NaN,NaN
6,Fairway Independent Mortgage,NaN,0.979745,NaN,0.000000,0.000000,-0.128324,0.000000,NaN,NaN


In [49]:
# load models
loaded_model_boa = pickle.load(open('boa_model.pkl', 'rb'))
loaded_model_wells = pickle.load(open('wells_model2.pkl', 'rb'))
loaded_model_chase = pickle.load(open('chase_model.pkl', 'rb'))
loaded_model_USB = pickle.load(open('usb_model.pkl', 'rb'))
loaded_model_LD = pickle.load(open('LD_model.pkl', 'rb'))
loaded_model_fair = pickle.load(open('fair_model.pkl', 'rb'))
loaded_model_cal = pickle.load(open('cal_model.pkl', 'rb'))

In [55]:
# Make probabalistic predictions for each bank and store in dict
predict_dict = {}
model_lst = [loaded_model_boa, loaded_model_wells, loaded_model_chase, loaded_model_USB, loaded_model_LD, 
            loaded_model_cal, loaded_model_fair]
for bank in bank_performance['Lender']: 
    for model in model_lst: 
        predict_dict[bank] = model.predict_proba_(example)

AttributeError: 'Pipeline' object has no attribute 'predict_proba_'

# Compatability Issues
* Because I wanted to first optimize each model, each dataframe that is passed in is potentially a different shape. I wanted to use the same thresholds for making decisions regarding missing values, so some columns were dropped for some banks, others for other banks. This is definitely an issue that needs to be addressed. However, to get a MVP web app, I'm going to pull example records (from the same index) from each model dataset and predict manually, for now. - this won't work because it is not the same applicant across all datasets. 
* I'm going to take one sample from one dataset, prior to any data restructuring. 
    * Then, I'll create a one-row dataframe of that sample, and pass it through the preprocessing steps for each bank.
    * Then, I'll store the resulting dataframe as an array for each bank, and use it to predict probabilities. 

In [69]:
loaded_model_wells.predict_proba(example.reshape(1,-1))[:,1]

array([1.])

In [ ]:
# User input example

def user_input(): 
    user_arr = []
    home_address = input('Property address: ')
    conforming = input('Is this a conforming loan? ')
    prop_val = input('Estimated home price: ')
    loan_amount = input('Estimated loan amount: ')
    loan_term = input('Length of loan in years: ')
    
    user_arr.append(prop_val)
    

In [22]:
# User info for web app demo (used row 42 in each )

In [77]:
%store -r JP_features
%store -r wells_features
%store -r features



In [79]:
print(len(JP_features))
print(len(wells_features))
print(len(features))

44
46
44


In [76]:
cal_features

,features,coefs
0,features,0.217958
1,coefs,-0.129080


In [74]:
for i in JP_features.features.values: 
    if i not in wells_features.features.values:
        print(i)

intro_rate_period
negative_amortization
balloon_payment
other_nonamortizing_features


In [80]:
wells_features.sort_values(by='coefs', ascending = True)

,features,coefs
44,aus-1_5,-9.094887
35,loan_type_2,-2.834129
43,aus-1_4,-0.784109
4,debt_to_income_ratio,-0.563385
37,loan_type_4,-0.422907
25,derived_race_Black or African American,-0.058048
16,conforming_loan_limit_C,-0.052168
24,derived_race_Asian,0.000000
26,derived_race_Free Form Text Only,0.000000
27,derived_race_Joint,0.000000
